In [2]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time

import time 
from scipy import stats
from matplotlib import pyplot

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_absolute_percentage_error as mape


In [3]:
import matplotlib as mpl
from matplotlib import pyplot
import matplotlib.pyplot as plt
mpl.rcParams['font.family'] = 'sans-serif'
def set_violin_color(vp,color,mediancolor):
    plt.setp(vp['bodies'], facecolor=color, edgecolor="white",alpha=0.7)
    plt.setp(vp['cbars'], color = color)
    plt.setp(vp['cmins'], color = "black")
    plt.setp(vp['cmaxes'], color = "black")
    plt.setp(vp['cmeans'], color = "black")
    plt.setp(vp['cmedians'],color = mediancolor)

def make_violin_plot(dataframe,col0,mediancolor,legendlabel,diff):
    stepepi_master0 = dataframe
    bp0 = plt.violinplot(stepepi_master0, positions=np.array(range(len(stepepi_master0)))*3.0+diff,showmeans = True,showmedians=True)
    positions=np.array(range(len(stepepi_master0)))
    set_violin_color(bp0,col0,mediancolor)
    plt.plot([], c=col0, label=legendlabel)
    plt.legend(loc='upper left') 
    return positions

import seaborn as sns
colorlist1 = sns.color_palette("bright").as_hex()
sns.color_palette("bright")

from sklearn.metrics import r2_score

pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-pa.pkl")

##### compile the well numbers for each species for its presence 

species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 
    
    
    
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A = pd.read_pickle("~/bge-analysis-simv3/A.pkl")

tableg=A.T
species_onesall = [] 
for species in range(16):
    species_onesall.append(tableg[tableg[species]==1.0][species].index.to_list())
species_zerosall = []
for species in range(16):
    species_zerosall.append(tableg[tableg[species]==0.0][species].index.to_list())

In [4]:
steadystate = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-sst-REL.pkl")
sst = steadystate.T

In [5]:
bias_var_listm = [3,3,3,3,3,3,3,3,5,3,5,3,3,7,5,7]

In [14]:
max_depthl = [2,5,10,15,20]
ntrees = [100]
max_featuresl = ["auto","sqrt","log2",0.33,0.5]

In [15]:
rdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/BENCHMARKING/RANDOM FOREST/OV-16splist1-v2/RF-16sp-OV-REL-5runs.pkl")
rdf.head(1)

,species,run idx,k-fold,k-fold idx,max depth,trees,feats,ypred,yact,train data,te data
0,0,0,3,0,2,100,auto,"[0.8001309889968298, 0.33078721118560517, 0.76...","[0.3391491238628783, 0.16302582467502283, 0.18...","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...","[1, 260, 4226, 4228, 4256, 8197, 24581]"


In [16]:
rdf2 = pd.read_pickle("~/compressed_sensingv1/realdatasets/BENCHMARKING/RANDOM FOREST/OV-16splist1-v2/RF-16sp-OV-REL-5runs-featsarefrac.pkl")
rdf2.head(1)

,species,run idx,k-fold,k-fold idx,max depth,trees,feats,ypred,yact,train data,te data
0,0,0,3,0,2,100,0.33,"[0.5875578557735204, 0.34797326000687157, 0.54...","[0.3391491238628783, 0.16302582467502283, 0.18...","[0, 2, 4, 20, 32, 128, 132, 4102, 4230, 8199, ...","[1, 260, 4226, 4228, 4256, 8197, 24581]"


In [17]:
rdf3 =  pd.concat([rdf,rdf2])

In [18]:
crossval = [] 
for runidx in range(5):
    errdf = rdf3[rdf3["run idx"]==runidx] 
    for speciesc in range(16):
        species = speciesc 
        sp1 = errdf[errdf["species"]==speciesc]
        subsetall = A[species_onesall[species]].T
        subsetall["indexinwhole"] = subsetall.index
        subsetall.index = np.arange(0,2**15,1)
        givendata = subsetall[subsetall["indexinwhole"].isin(species_ones[species])]
        sstgiven = steadystate.T[subsetall[subsetall["indexinwhole"].isin(species_ones[species])]["indexinwhole"].values].T
        sstgiven.index = givendata.index
        m = bias_var_listm[speciesc]
        for md in max_depthl:
            rho1 = sp1[sp1["max depth"]==md]
            for nest in ntrees: 
                ntre = rho1[rho1["trees"]==nest]
                for mfeats in max_featuresl:
                    mdf = ntre[ntre["feats"]==mfeats]
                    #print(mdf["ypred"].iloc[0].shape)
                    r2 = [] 
                    mape2 = [] 
                    yallbest  = []
                    yallact  = [] 
                    for iterii in range(m):
                        idx = mdf["te data"].iloc[iterii]
                        #idx.sort()
                        yact = sstgiven.T[idx].T[species].values
                        ybest = mdf["ypred"].iloc[iterii]
                        ybestdf1 = pd.DataFrame(ybest)
                        ybestdf1[ybestdf1[0]<1e-10]=0
                        slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                        rval = r2_score(yact,ybestdf1[0].values)
                        gop = 1/(2-rval)
                        r2.append([gop,r2_score(yact,ybestdf1[0].values)])
                        mape2.append(mape(yact,ybestdf1[0].values))
                        yallbest.append(ybestdf1[0].values)
                        yallact.append(yact)
                        slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                        rval = r2_score(yact,ybestdf1[0].values)
                        gop = 1/(2-rval)
                        mape1 = mape(yact,ybestdf1[0].values)
                        r2.append([gop,rval,mape1,slope,intercept,r_value**2])
                    r2df = pd.DataFrame(r2)
                    if m == 2:
                        ybestagg = [*yallbest[0],*yallbest[1]]
                        yactagg = [*yallact[0],*yallact[1]]
                    if m == 3:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2]]
                    if m == 5:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
                    if m == 6:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5]]
                    if m == 7:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
                    if m == 9:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
                    slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
                    rval2 = r2_score(yactagg,ybestagg)
                    gop2 = 1/(2-rval2)
                    mean1 = r2df.mean().values[0]
                    mean2 = r2df.mean().values[1]
                    mean3 = r2df.mean().values[2]
                    mean4 = r2df.mean().values[3]
                    mean5 = r2df.mean().values[4]
                    mean6 = r2df.mean().values[5]
                    mape2 = mape(yactagg,ybestagg)
                    crossval.append([runidx,m,species,md,nest,mfeats,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])

In [19]:
crossdf = pd.DataFrame(crossval,columns=["runidx","k-fold","sp","max_depth","trees","features","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [20]:
crossdf.head(2)

,runidx,k-fold,sp,max_depth,trees,features,nnseagg,r2agg,mapeagg,slagg,...,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
0,0,3,0,2,100,auto,0.465888,-0.146438,2.234871,0.180939,...,0.081141,break,0.377401,-4.076133,2.234871,0.830124,0.249618,0.399282,"[0.3391491238628783, 0.16302582467502283, 0.18...","[0.8001309889968298, 0.33078721118560517, 0.76..."
1,0,3,0,2,100,sqrt,0.495936,-0.016390,1.927708,0.045169,...,0.021433,break,0.383228,-1.833576,1.927708,0.398951,0.258005,0.308110,"[0.3391491238628783, 0.16302582467502283, 0.18...","[0.557409771577533, 0.3732736235409721, 0.4609..."


In [21]:
pd.to_pickle(crossdf,"~/compressed_sensingv1/realdatasets/BENCHMARKING/RANDOM FOREST/OV-16splist1-v2/RF-REL-analysis-fracfeats.pkl")
#pd.to_pickle(crosslindf,"sigma-REL-analysis.pkl")

In [23]:
crossdf.head(3)

,runidx,k-fold,sp,max_depth,trees,features,nnseagg,r2agg,mapeagg,slagg,...,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
0,0,3,0,2,100,auto,0.465888,-0.146438,2.234871,0.180939,...,0.081141,break,0.377401,-4.076133,2.234871,0.830124,0.249618,0.399282,"[0.3391491238628783, 0.16302582467502283, 0.18...","[0.8001309889968298, 0.33078721118560517, 0.76..."
1,0,3,0,2,100,sqrt,0.495936,-0.016390,1.927708,0.045169,...,0.021433,break,0.383228,-1.833576,1.927708,0.398951,0.258005,0.308110,"[0.3391491238628783, 0.16302582467502283, 0.18...","[0.557409771577533, 0.3732736235409721, 0.4609..."
2,0,3,0,2,100,log2,0.532792,0.123094,1.760097,0.118451,...,0.131150,break,0.416157,-1.151544,1.760097,0.416707,0.237095,0.454701,"[0.3391491238628783, 0.16302582467502283, 0.18...","[0.5282648025757541, 0.3508087671959769, 0.386..."
